In [1]:
import cv2
import numpy as np
from pdf2image import convert_from_path


def detect_text_regions(image):
    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # MSERを使用してテキスト領域を検出
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray)

    # 検出された領域をマージ
    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]

    # 小さすぎる領域を除外
    hulls = [h for h in hulls if cv2.contourArea(h) > 0]

    # 領域をマージ
    mask = np.zeros((image.shape[0], image.shape[1], 1), dtype=np.uint8)
    for hull in hulls:
        cv2.drawContours(mask, [hull], 0, (255), -1)

    # 膨張と収縮を行ってテキスト領域をつなげる
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.dilate(mask, kernel, iterations=5)
    mask = cv2.erode(mask, kernel, iterations=4)

    # 輪郭を検出
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # バウンディングボックスを取得
    boxes = [cv2.boundingRect(c) for c in contours]

    return boxes


def draw_bounding_boxes(image, boxes):
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return image


# 画像を読み込む
images = convert_from_path(r"../data/pdf/ir2024_all_ja.pdf")
image = cv2.cvtColor(np.array(images[1]), cv2.COLOR_RGB2BGR)

# テキスト領域を検出
boxes = detect_text_regions(image)

# 境界ボックスを描画
image_with_boxes = draw_bounding_boxes(image.copy(), boxes)

# 結果を表示
# cv2.imshow('Image with Bounding Boxes', image_with_boxes)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# 結果を保存（オプション）
cv2.imwrite("../data/output/image_with_bounding_boxes.jpg", image_with_boxes)

True